## Importanto dataset do moabb usando braindecode





In [ ]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
!pip install moabb braindecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1 MB 1.5 kB/s 
     |████████████████████████████████| 8.8 MB 10.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.2.0+cu92 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.2.0+cu92 which 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from braindecode import datasets
from braindecode.datasets import MOABBDataset

subject_id = [3]
dataset = MOABBDataset(dataset_name="BNCI2014001", subject_ids=subject_id)

48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]


In [ ]:
print(dataset.description)

    subject    session    run
0         3  session_T  run_0
1         3  session_T  run_1
2         3  session_T  run_2
3         3  session_T  run_3
4         3  session_T  run_4
5         3  session_T  run_5
6         3  session_E  run_0
7         3  session_E  run_1
8         3  session_E  run_2
9         3  session_E  run_3
10        3  session_E  run_4
11        3  session_E  run_5


## Pré-processamento

In [ ]:
from numpy import multiply
from braindecode.preprocessing import (
    exponential_moving_standardize, preprocess, Preprocessor, scale)

low_cut_hz = 4.  # low cut frequency for filtering
high_cut_hz = 38.  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Mantendo apenas os sinais de EEG
    Preprocessor(scale, factor=1e6, apply_on_array=True),  # Convertendo V para uV
    Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    #Preprocessor(exponential_moving_standardize,  # Exponential moving standardization
    #             factor_new=factor_new, init_block_size=init_block_size)
]

# Transform the data
preprocess(dataset, preprocessors)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function scale is deprecated; will be removed in 0.7.0. Use numpy.multiply instead.
  warnings.warn(msg, category=FutureWarning)


Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 sec)

Filteri

## Criando janela de eventos

In [ ]:
#import numpy as np
from braindecode.preprocessing import \
    create_windows_from_events, create_fixed_length_windows

trial_start_offset_seconds = -0.5
# Extract sampling frequency, check that they are same in all datasets
sfreq = dataset.datasets[0].raw.info['sfreq']
assert all([ds.raw.info['sfreq'] == sfreq for ds in dataset.datasets])
# Calculate the trial start offset in samples.
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Adding metadata with 4 columns
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1125 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Adding metadata with 4 columns
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1125 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Adding metadata with 4 columns
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1125 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Adding metadata with 4 columns
48 matching even

In [ ]:
windows_dataset.datasets[0].windows.to_data_frame()

,time,condition,epoch,Fz,FC3,FC1,FCz,FC2,FC4,C5,...,C6,CP3,CP1,CPz,CP2,CP4,P1,Pz,P2,POz
0,0,left_hand,0,-1.508807e+06,-2.057763e+06,-1.103649e+06,-2.219515e+06,-2.574769e+06,-3.144823e+06,7.023217e+05,...,-5.502411e+06,3.577957e+05,8.728321e+05,6.308310e+04,-1.888485e+05,-1.639667e+06,-4.184171e+05,1.075510e+06,3.261588e+05,9.336982e+05
1,4,left_hand,0,-1.082187e+06,-3.989230e+06,-2.268988e+06,-2.244667e+06,-1.589948e+06,-2.468012e+06,-2.240591e+06,...,-3.965960e+06,-1.987421e+06,-3.316957e+05,5.093878e+04,1.065941e+05,-1.610188e+06,-7.784045e+05,1.083369e+06,4.176927e+05,1.179934e+06
2,8,left_hand,0,1.277184e+06,-3.315700e+06,-1.041971e+06,8.258523e+04,1.672815e+06,4.358822e+05,-3.020771e+06,...,-1.409225e+06,-1.386379e+06,1.054109e+06,2.260364e+06,2.246835e+06,5.492965e+04,1.680642e+06,3.248685e+06,2.437184e+06,3.663755e+06
3,12,left_hand,0,4.524520e+06,-6.738757e+05,1.773174e+06,3.675256e+06,5.693913e+06,4.239017e+06,-2.123081e+06,...,9.952130e+05,1.007480e+06,3.632100e+06,5.231085e+06,4.907994e+06,2.149616e+06,5.145584e+06,6.058800e+06,4.940544e+06,6.811878e+06
4,16,left_hand,0,7.234125e+06,2.266647e+06,4.508278e+06,6.860393e+06,8.623931e+06,7.376947e+06,-1.052597e+06,...,2.694435e+06,3.023170e+06,5.366535e+06,7.196790e+06,6.614796e+06,3.534755e+06,7.320809e+06,7.771216e+06,6.411019e+06,8.681854e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53995,4480,tongue,47,9.111334e+06,1.377034e+06,1.638222e+05,4.269832e+06,4.103577e+06,5.799652e+06,-1.718842e+07,...,3.174204e+06,-1.958988e+07,-1.404527e+07,-7.900694e+06,-7.109345e+06,-5.044626e+06,-1.241805e+07,-1.109200e+07,-8.202974e+06,-8.980056e+06
53996,4484,tongue,47,-9.170075e+05,-1.095517e+07,-1.207084e+07,-7.872026e+06,-7.507321e+06,-4.450548e+06,-2.658675e+07,...,-3.734991e+06,-3.103178e+07,-2.602817e+07,-2.060095e+07,-1.880273e+07,-1.522003e+07,-2.244362e+07,-2.171100e+07,-1.862106e+07,-1.801376e+07
53997,4488,tongue,47,-9.684617e+06,-2.306477e+07,-2.274555e+07,-1.882145e+07,-1.750539e+07,-1.373456e+07,-3.035757e+07,...,-9.141157e+06,-3.795132e+07,-3.392993e+07,-3.071356e+07,-2.769745e+07,-2.308862e+07,-2.896700e+07,-2.905143e+07,-2.601397e+07,-2.394735e+07
53998,4492,tongue,47,-1.298013e+07,-2.834688e+07,-2.650257e+07,-2.325483e+07,-2.112429e+07,-1.740232e+07,-2.604530e+07,...,-9.682583e+06,-3.568819e+07,-3.319534e+07,-3.274284e+07,-2.883254e+07,-2.402006e+07,-2.741615e+07,-2.852338e+07,-2.592088e+07,-2.252372e+07


## Configurando GPU

In [ ]:
!nvidia-smi

Tue Apr 26 21:59:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()
# Output would be True if Pytorch is using GPU otherwise it would be False.

True

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()
# Standard output is '/device:GPU:0'

'/device:GPU:0'

## EEGNet


""" Written by, Sriram Ravindran, sriram@ucsd.edu

Original paper - https://arxiv.org/abs/1611.08024

Please reach out to me if you spot an error. """

In [ ]:
sensores = ['Fz','FC3', 'FCz', 'FC2', 'FC4', 'C3', 'C1', 'Cz',
       'C2','C4', 'CP1', 'CPz', 'CP2', 'CP4', 'P1', 'Pz', 'P2','POz']

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 64), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4*2*7, 1)
        

    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)
        
        # FC Layer
        x = x.reshape(-1, 4*2*7)
        x = F.sigmoid(self.fc1(x))
        return x


net = EEGNet().cuda(0)
print(net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)).cuda(0))))
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())

tensor([[0.5861]], device='cuda:0', grad_fn=<SigmoidBackward0>)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range(len(X)/batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]).cuda(0))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X).cuda(0))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

In [ ]:
X_train = treino[sensores].to_numpy()
y_train = treino['condition'].to_numpy()

X_val = teste[sensores].to_numpy()
y_val = teste['condition'].to_numpy()

X_test = teste[sensores].to_numpy()
y_test = teste['condition'].to_numpy

In [ ]:
len(X_train)

6744

In [ ]:
y_train

array([0, 0, 0, ..., 1, 1, 1], dtype=int8)

In [ ]:
X_train

array([[0.16260997, 0.01887873, 0.1503982 , ..., 0.3015323 , 0.26041265,
        0.11692117],
       [0.21601941, 0.26428911, 0.30649206, ..., 0.27136895, 0.20783382,
        0.14008009],
       [0.16634103, 0.40014655, 0.26929162, ..., 0.2592646 , 0.28985286,
        0.3714392 ],
       ...,
       [0.06554597, 0.0585961 , 0.08644259, ..., 0.3347977 , 0.39156665,
        0.47225502],
       [0.32470892, 0.14404333, 0.29194229, ..., 0.26457471, 0.32472721,
        0.52009396],
       [0.07634172, 0.22652135, 0.09636481, ..., 0.30733174, 0.23362646,
        0.36701343]])

In [ ]:
batch_size = 32

for epoch in range(10):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    
    running_loss = 0.0
    for i in range(len(X_train)//batch_size-1):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_train[s:e])
        labels = torch.FloatTensor(np.array([y_train[s:e]]).T*1.0)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.data[0]
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print(params)
    print("Training Loss ", running_loss)
    print("Train - ", evaluate(net, X_train, y_train, params))
    print("Validation - ", evaluate(net, X_val, y_val, params))
    print("Test - ", evaluate(net, X_test, y_test, params))


Epoch  0


RuntimeError: ignored

#Explicadores de modelo caixa preta


##PDP

##ALE

##Protótipos

##ICE


##Counterfactual


##SHAP ou IG